In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data= pd.read_csv("../input/mobile-price-classification/train.csv")

In [ ]:
data.head()

In [ ]:
data.info()
data.count()

There's no missing value. So we can start with EDA specifically Feature Engineering

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data['price_range'].describe()
corrmat = data.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

From the heatmap, we observe that there are a three tuples that are corelated-
1) 3G and 4G
2) pc and fc 
3) px_width and px_height

As a user, the differentiating factor would be fc and 4G. Because if we have a good front camera it means that primary camera will also be better or atleast similar. If a phone is having 4G, it will definately have 3G. So, the differentiating factors would be 4G and fc. As far as pixcel is considered, we can select either of them

In [ ]:
data.drop(columns= ["three_g",'four_g',"m_dep","n_cores","sc_h","pc","wifi","blue"],axis=1, inplace=True)

In [ ]:
print("No. of touch screen phones is %f",sum(data['touch_screen']==1))


We observe that both touch screen and not touch screen phones are present in the dataset, still it is not corelated to the price range. Hence we will remove it. Similarly, based on heatmap, we remove mobile weight and dock speed

In [ ]:
data.drop(columns=['touch_screen','mobile_wt','clock_speed'],axis=1, inplace=True)

In [ ]:
data

In [ ]:
y= data.iloc[:,-1]
x= data.iloc[:,0:9]

In [ ]:
x

****Detecting and Removing Outliers****

In [ ]:
import seaborn as sns
x.boxplot(figsize=(14,8))

There are Outliers in fc and px_height. So applying rescaling and normalization

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)

In [ ]:
type(x)

In [ ]:
y= y.to_numpy()
type(y)

In [ ]:
y= y.reshape(-1,1)
from sklearn.preprocessing import OneHotEncoder
ohot = OneHotEncoder()
y = ohot.fit_transform(y)

In [ ]:
y= y.toarray()
y

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Flatten,Dense
from keras.models import Sequential

In [ ]:
model=Sequential()
model.add(Dense(10, activation='relu', input_dim=9 ))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='softmax'))
model.summary()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [ ]:
history = model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test), batch_size=32)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train','Validation Set'],loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train','Validation Set'],loc='upper left')
plt.show()

In [ ]:
y_pred = model.predict(x_test)

#lets do the inverse one hot encoding
pred = []
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
    
# also inverse encoding for y_test labels

test = []
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))

In [ ]:

from sklearn.metrics import accuracy_score
acc = accuracy_score(pred,test)
print("Accuracy of Your Model is = " + str(acc*100))

In [ ]:
from tensorflow.keras.models import save_model, load_model
filepath = './input/saved_model'
save_model(model, filepath)

In [ ]:
test = pd.read_csv("../input/mobile-price-classification/test.csv")
test.drop(columns= ["id","three_g",'four_g',"m_dep","n_cores","sc_h","pc","wifi","blue",'touch_screen','mobile_wt','clock_speed'],axis=1, inplace=True)

In [ ]:
test.head()

In [ ]:
test.info()
test.isna().sum()

No null Values in test Set

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
test = sc.fit_transform(test)

In [ ]:
type(test)

In [ ]:
model= load_model(filepath, compile=True)

In [ ]:
predictions = model.predict(test)


In [ ]:
Price_range = []
for i in range(len(predictions)):
    Price_range.append(np.argmax(predictions[i]))

In [ ]:
Price_range


In [ ]:
my_sub = pd.DataFrame({'PriceRange':Price_range})
my_sub.to_csv('submission.csv', index=True)